In [ ]:
import pdb
import os
import gc
import random
import glob
import sys
import random
import scipy.ndimage
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from imutils import paths
from itertools import chain, islice
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.applications.inception_v3 import InceptionV3
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [ ]:
source_path = list(paths.list_images(SOURCE_DIR))
target_path = list(paths.list_images(TARGET_DIR))

labels_source = [p.split(os.path.sep)[-2] for p in source_path]
labels_target = [p.split(os.path.sep)[-2] for p in target_path]

In [ ]:
classes = ['aeroplane', 'bicycle', 'bus', 'car', 'horse', 'knife', 'motorcycle', 'person', 'plant', 'skateboard', 
           'train', 'truck']

In [ ]:
def paths_(label, path):
    
    return np.array([x for x in path if '%s' %label in x])

In [ ]:
def data_count(x):
    
    xid, xcount = np.unique(x, return_counts=True)
    
    return xid, xcount

In [ ]:
def get_data_min(label):
    
    id_, count = data_count(label)
    min_count = min(count)
    min_id = np.argmin(count)
    label_min = id_[min_id]
    
    return min_count, label_min

In [ ]:
source_data = get_data_min(labels_source)
target_data = get_data_min(labels_target)

In [ ]:
source_data

In [ ]:
target_data

In [ ]:
source_aeroplane = np.random.choice(paths_('aeroplane', source_path), size = source_data[0])
source_bicycle = paths_('bicycle', source_path)
source_bus = np.random.choice(paths_('bus', source_path), size = source_data[0])
source_car = np.random.choice(paths_('car', source_path), size = source_data[0])
source_horse = np.random.choice(paths_('horse', source_path), size = source_data[0])
source_knife = np.random.choice(paths_('knife', source_path), size = source_data[0])
source_motorcycle = np.random.choice(paths_('motorcycle', source_path), size = source_data[0])
source_person = np.random.choice(paths_('person', source_path), size = source_data[0])
source_plant = np.random.choice(paths_('plant', source_path), size = source_data[0])
source_skateboard = np.random.choice(paths_('skateboard', source_path), size = source_data[0])
source_train = np.random.choice(paths_('train/train', source_path), size = source_data[0])
source_truck = np.random.choice(paths_('truck', source_path), size = source_data[0])

In [ ]:
source_path_ = list(chain(source_aeroplane, source_bicycle, source_bus, source_car, source_horse, source_knife, 
                     source_motorcycle, source_person, source_plant, source_skateboard, source_train, source_truck))

In [ ]:
target_aeroplane = np.random.choice(paths_('aeroplane', target_path), size = target_data[0])
target_bicycle = np.random.choice(paths_('bicycle', target_path), size = target_data[0])
target_bus = np.random.choice(paths_('bus', target_path), size = target_data[0])
target_car = np.random.choice(paths_('car', target_path), size = target_data[0])
target_horse = np.random.choice(paths_('horse', target_path), size = target_data[0])
target_knife = paths_('knife', target_path)
target_motorcycle = np.random.choice(paths_('motorcycle', target_path), size = target_data[0])
target_person = np.random.choice(paths_('person', target_path), size = target_data[0])
target_plant = np.random.choice(paths_('plant', target_path), size = target_data[0])
target_skateboard = np.random.choice(paths_('skateboard', target_path), size = target_data[0])
target_train = np.random.choice(paths_('train', target_path), size = target_data[0])
target_truck = np.random.choice(paths_('truck', target_path), size = target_data[0])

In [ ]:
target_path_ = list(chain(target_aeroplane, target_bicycle, target_bus, target_car, target_horse, target_knife, 
                     target_motorcycle, target_person, target_plant, target_skateboard, target_train, target_truck))

In [ ]:
source_y = [p.split(os.path.sep)[-2] for p in source_path_]
target_y = [p.split(os.path.sep)[-2] for p in target_path_]

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(source_y)
labels_t = le.fit_transform(target_y)

In [ ]:
len(source_path_), len(target_path_)

In [ ]:
labels.shape, labels_t.shape

In [ ]:
del source_y, target_y
gc.collect()

In [ ]:
labels_unique = np.load(os.path.join(DATA_LOAD_DIR, 'labels.npy'))

In [ ]:
clf = linear_model.SGDClassifier(max_iter = 1000, tol = 1e-4, n_jobs = -1, loss = 'log')

In [ ]:
def model_create(transfer_model):
    if transfer_model == 'InceptionV3':
        with tf.device('/gpu:1'):
            return InceptionV3(weights = 'imagenet', include_top = False)
    elif transfer_model == 'VGG16':
        with tf.device('/gpu:1'):
            return VGG16(weights = 'imagenet', include_top = False)
    elif transfer_model == 'VGG19':
        with tf.device('/gpu:1'):
            return VGG19(weights = 'imagenet', include_top = False)
    elif transfer_model == 'Xception':
        with tf.device('/gpu:1'):
            return Xception(weights = 'imagenet', include_top = False)
    elif transfer_model == 'DenseNet121':
        with tf.device('/gpu:1'):
            return DenseNet121(weights = 'imagenet', include_top = False)

In [ ]:
def model_reshape(transfer_model, features):
    if transfer_model == 'InceptionV3':
        return features.reshape((features.shape[0], 5 * 5 * 2048))
    elif transfer_model == 'VGG16':
        return features.reshape((features.shape[0], 7 * 7 * 512))
    elif transfer_model == 'VGG19':
        return features.reshape((features.shape[0], 7 * 7 * 512))
    elif transfer_model == 'Xception':
        return features.reshape((features.shape[0], 7 * 7 * 2048))
    elif transfer_model == 'DenseNet121':
        return features.reshape((features.shape[0], 7 * 7 * 1024))

In [ ]:
def model_train(i, model, features, label):
    
    print('Training batch: ', i)
    model.partial_fit(features, label, classes = labels_unique)
       
    return model

In [ ]:
def target_pred(model, target_features):
    
    #print('Predicting batch features')
    return model.predict(target_features)

In [ ]:
step_size = 128

batches = int(len(source_path_) / step_size)
batches_target = int(len(target_path_) / step_size)

In [ ]:
def source_batch_gen(batch, label):
    
    val = int(batch / len(classes))
        
    for batch in range(batches):
        
        if batch < (batches - 1):
            
            batch_aeroplane = source_aeroplane[batch * val: (batch + 1) * val]
            batch_bicycle = source_bicycle[batch * val: (batch + 1) * val]
            batch_bus = source_bus[batch * val: (batch + 1) * val]
            batch_car = source_car[batch * val: (batch + 1) * val]
            batch_horse = source_horse[batch * val: (batch + 1) * val]
            batch_knife = source_knife[batch * val: (batch + 1) * val]
            batch_motorcycle = source_motorcycle[batch * val: (batch + 1) * val]
            batch_person = source_person[batch * val: (batch + 1) * val]
            batch_plant = source_plant[batch * val: (batch + 1) * val]
            batch_skateboard = source_skateboard[batch * val: (batch + 1) * val]
            batch_train = source_train[batch * val: (batch + 1) * val]
            batch_truck = source_truck[batch * val: (batch + 1) * val]

            source_ = list(chain(batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, 
                                 batch_motorcycle, batch_person, batch_plant, batch_skateboard, batch_train, batch_truck))

            aeroy = label[:7365]
            batch_aeroplane_y = aeroy[batch * val: (batch + 1) * val]

            biy = label[7365: 14730]
            batch_bicycle_y = biy[batch * val: (batch + 1) * val]

            busy = label[14730: 22095]
            batch_bus_y = busy[batch * val: (batch + 1) * val]

            cary = label[22095: 29460]
            batch_car_y = cary[batch * val: (batch + 1) * val]

            horsey = label[29460: 36825]
            batch_horse_y = horsey[batch * val: (batch + 1) * val]

            knifey = label[36825: 44190]
            batch_knife_y = knifey[batch * val: (batch + 1) * val]

            motory = label[44190: 51555]
            batch_motorcycle_y = motory[batch * val: (batch + 1) * val]

            pery = label[51555: 58920]
            batch_person_y = pery[batch * val: (batch + 1) * val]

            planty = label[58920: 66285]
            batch_plant_y = planty[batch * val: (batch + 1) * val]

            skatey = label[66285: 73650]
            batch_skateboard_y = skatey[batch * val: (batch + 1) * val]

            trainy = label[73650: 81015]
            batch_train_y = trainy[batch * val: (batch + 1) * val]

            trucky = label[81015:]
            batch_truck_y = trucky[batch * val: (batch + 1) * val]

            source_lab = list(chain(batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, batch_horse_y, batch_knife_y, 
                                 batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
                                 batch_truck_y))


            yield (source_, source_lab)
            
        elif batch == (batches - 1):
            
            batch_aeroplane = source_aeroplane[batch * val:]
            batch_bicycle = source_bicycle[batch * val:]
            batch_bus = source_bus[batch * val:]
            batch_car = source_car[batch * val:]
            batch_horse = source_horse[batch * val:]
            batch_knife = source_knife[batch * val:]
            batch_motorcycle = source_motorcycle[batch * val:]
            batch_person = source_person[batch * val:]
            batch_plant = source_plant[batch * val:]
            batch_skateboard = source_skateboard[batch * val:]
            batch_train = source_train[batch * val:]
            batch_truck = source_truck[batch * val:]

            source_ = list(chain(batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, 
                                 batch_motorcycle, batch_person, batch_plant, batch_skateboard, batch_train, batch_truck))

            aeroy = label[:7365]
            batch_aeroplane_y = aeroy[batch * val:]

            biy = label[7365: 14730]
            batch_bicycle_y = biy[batch * val:]

            busy = label[14730: 22095]
            batch_bus_y = busy[batch * val:]

            cary = label[22095: 29460]
            batch_car_y = cary[batch * val:]

            horsey = label[29460: 36825]
            batch_horse_y = horsey[batch * val:]

            knifey = label[36825: 44190]
            batch_knife_y = knifey[batch * val:]

            motory = label[44190: 51555]
            batch_motorcycle_y = motory[batch * val:]

            pery = label[51555: 58920]
            batch_person_y = pery[batch * val:]

            planty = label[58920: 66285]
            batch_plant_y = planty[batch * val:]

            skatey = label[66285: 73650]
            batch_skateboard_y = skatey[batch * val:]

            trainy = label[73650: 81015]
            batch_train_y = trainy[batch * val:]

            trucky = label[81015:]
            batch_truck_y = trucky[batch * val:]

            source_lab = list(chain(batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, batch_horse_y, batch_knife_y, 
                                 batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
                                 batch_truck_y))


            yield (source_, source_lab)

In [ ]:
def target_batch_gen(n, pred_): 
    
    val = int(n / len(classes))
    
    if type(pred_) == np.ndarray:
        pred1 = pred_.tolist()

    for batch in range(batches_target):
        
        if batch < (batches_target - 1):
            
            print('Target batch number for prediction: ', batch)
            batch_aeroplane = target_aeroplane[batch * val: (batch + 1) * val]
            batch_bicycle = target_bicycle[batch * val: (batch + 1) * val]
            batch_bus = target_bus[batch * val: (batch + 1) * val]
            batch_car = target_car[batch * val: (batch + 1) * val]
            batch_horse = target_horse[batch * val: (batch + 1) * val]
            batch_knife = target_knife[batch * val: (batch + 1) * val]
            batch_motorcycle = target_motorcycle[batch * val: (batch + 1) * val]
            batch_person = target_person[batch * val: (batch + 1) * val]
            batch_plant = target_plant[batch * val: (batch + 1) * val]
            batch_skateboard = target_skateboard[batch * val: (batch + 1) * val]
            batch_train = target_train[batch * val: (batch + 1) * val]
            batch_truck = target_truck[batch * val: (batch + 1) * val]

            target_ = list(chain(batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, 
                                 batch_motorcycle, batch_person, batch_plant, batch_skateboard, batch_train, batch_truck))

            if type(pred_) == np.ndarray:
                aeroy = pred_[:2075]
                batch_aeroplane_y = aeroy[batch * val: (batch + 1) * val]

                biy = pred_[2075: 4150]
                batch_bicycle_y = biy[batch * val: (batch + 1) * val]

                busy = pred_[4150: 6225]
                batch_bus_y = busy[batch * val: (batch + 1) * val]

                cary = pred_[6225: 8300]
                batch_car_y = cary[batch * val: (batch + 1) * val]

                horsey = pred_[8300: 10375]
                batch_horse_y = horsey[batch * val: (batch + 1) * val]

                knifey = pred_[10375: 12450]
                batch_knife_y = knifey[batch * val: (batch + 1) * val]

                motory = pred_[12450: 14525]
                batch_motorcycle_y = motory[batch * val: (batch + 1) * val]

                pery = pred_[14525: 16600]
                batch_person_y = pery[batch * val: (batch + 1) * val]

                planty = pred_[16600: 18675]
                batch_plant_y = planty[batch * val: (batch + 1) * val]

                skatey = pred_[18675: 20750]
                batch_skateboard_y = skatey[batch * val: (batch + 1) * val]

                trainy = pred_[20750: 22825]
                batch_train_y = trainy[batch * val: (batch + 1) * val]

                trucky = pred_[22825:]
                batch_truck_y = trucky[batch * val: (batch + 1) * val]

                target_lab = list(chain(batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, batch_horse_y, batch_knife_y, 
                                     batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
                                     batch_truck_y))


                yield (target_, target_lab)

            elif pred_ == None:
                yield target_
                
        
        if batch == (batches_target - 1):
            
            print('Target batch number for prediction: ', batch)
            batch_aeroplane = target_aeroplane[batch * val:]
            batch_bicycle = target_bicycle[batch * val:]
            batch_bus = target_bus[batch * val:]
            batch_car = target_car[batch * val:]
            batch_horse = target_horse[batch * val:]
            batch_knife = target_knife[batch * val:]
            batch_motorcycle = target_motorcycle[batch * val:]
            batch_person = target_person[batch * val:]
            batch_plant = target_plant[batch * val:]
            batch_skateboard = target_skateboard[batch * val:]
            batch_train = target_train[batch * val:]
            batch_truck = target_truck[batch * val:]

            target_ = list(chain(batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, 
                                 batch_motorcycle, batch_person, batch_plant, batch_skateboard, batch_train, batch_truck))

            if type(pred_) == np.ndarray:
                aeroy = pred_[:2075]
                batch_aeroplane_y = aeroy[batch * val:]

                biy = pred_[2075: 4150]
                batch_bicycle_y = biy[batch * val:]

                busy = pred_[4150: 6225]
                batch_bus_y = busy[batch * val:]

                cary = pred_[6225: 8300]
                batch_car_y = cary[batch * val:]

                horsey = pred_[8300: 10375]
                batch_horse_y = horsey[batch * val:]

                knifey = pred_[10375: 12450]
                batch_knife_y = knifey[batch * val:]

                motory = pred_[12450: 14525]
                batch_motorcycle_y = motory[batch * val:]

                pery = pred_[14525: 16600]
                batch_person_y = pery[batch * val:]

                planty = pred_[16600: 18675]
                batch_plant_y = planty[batch * val:]

                skatey = pred_[18675: 20750]
                batch_skateboard_y = skatey[batch * val:]

                trainy = pred_[20750: 22825]
                batch_train_y = trainy[batch * val:]

                trucky = pred_[22825:]
                batch_truck_y = trucky[batch * val:]

                target_lab = list(chain(batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, batch_horse_y, batch_knife_y, 
                                     batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
                                     batch_truck_y))


                yield (target_, target_lab)

            elif pred_ == None:
                yield target_

In [ ]:
def target_sample_extract(n, pred_):
    
    val = int(n / len(classes))
    
    if type(pred_) == np.ndarray:
        pred1 = pred_.tolist()
        
    batch_aeroplane = target_aeroplane[:val]
    batch_bicycle = target_bicycle[:val]
    batch_bus = target_bus[:val]
    batch_car = target_car[:val]
    batch_horse = target_horse[:val]
    batch_knife = target_knife[:val]
    batch_motorcycle = target_motorcycle[:val]
    batch_person = target_person[:val]
    batch_plant = target_plant[:val]
    batch_skateboard = target_skateboard[:val]
    batch_train = target_train[:val]
    batch_truck = target_truck[:val]
    
    target_ = list(chain(batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, 
                                 batch_motorcycle, batch_person, batch_plant, batch_skateboard, batch_train, batch_truck))
    
    aeroy = pred_[:2075]
    batch_aeroplane_y = aeroy[:val]

    biy = pred_[2075: 4150]
    batch_bicycle_y = biy[:val]

    busy = pred_[4150: 6225]
    batch_bus_y = busy[:val]

    cary = pred_[6225: 8300]
    batch_car_y = cary[:val]

    horsey = pred_[8300: 10375]
    batch_horse_y = horsey[:val]

    knifey = pred_[10375: 12450]
    batch_knife_y = knifey[:val]

    motory = pred_[12450: 14525]
    batch_motorcycle_y = motory[:val]

    pery = pred_[14525: 16600]
    batch_person_y = pery[:val]

    planty = pred_[16600: 18675]
    batch_plant_y = planty[:val]

    skatey = pred_[18675: 20750]
    batch_skateboard_y = skatey[:val]

    trainy = pred_[20750: 22825]
    batch_train_y = trainy[:val]

    trucky = pred_[22825:]
    batch_truck_y = trucky[:val]
    
    target_lab = list(chain(batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, batch_horse_y, batch_knife_y, 
                            batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
                            batch_truck_y))
    
    assert(len(target_) == len(target_lab))
    
    del batch_aeroplane, batch_bicycle, batch_bus, batch_car, batch_horse, batch_knife, batch_motorcycle, batch_person, 
    batch_plant, batch_skateboard, batch_train, batch_truck, batch_aeroplane_y, batch_bicycle_y, batch_bus_y, batch_car_y, 
    batch_horse_y, batch_knife_y, batch_motorcycle_y, batch_person_y, batch_plant_y, batch_skateboard_y, batch_train_y, 
    batch_truck_y
    
    gc.collect()
    
    return target_, target_lab

In [ ]:
def feature_train(batchPaths, model1, model_, source, iteration, b, model_partial, batchLabels):
    
    batchImages = []
    #print(len(batchPaths))
    for imagePath in batchPaths:

        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)

        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)

        batchImages.append(image)

    batchImages = np.vstack(batchImages)

    features = model1.predict(batchImages, batch_size = step_size)
    features1 = model_reshape(model_, features)
    
    del image, batchImages
    gc.collect()

    if source == 'source' and b == 0:
        return model_train(b, clf, features1, batchLabels)
    elif source == 'source' and b != 0:
        return model_train(b, model_partial, features1, batchLabels)
    elif source == 'target' and iteration == 0:
        return target_pred(model_partial, features1)
    elif source == 'target' and b == 0 and iteration != 0:
        return model_train(b, clf, features1, batchLabels)
    elif source == 'target' and b != 0 and iteration != 0:
        return model_train(b, model_partial, features1, batchLabels)

In [ ]:
def feature_extract(batch_size, pseudo_labels, source_labels, model_, source, trained_model, iteration, n_k):
    
    pred = []
    s_path = []
    
    model1 = model_create(model_)

    if source == 'target' and iteration == 0:
        
        for target_sample in target_batch_gen(batch_size, None):

                pred.append(feature_train(target_sample, model1, model_, source, iteration, None, trained_model, None))
    
    elif trained_model == None and n_k == None:
        
        b = 0
        
        for batchPaths, source_labels_ in source_batch_gen(batch_size, source_labels):
            
            batchLabels = np.array(source_labels_)
            
            if b == 0:
                model_partial = feature_train(batchPaths, model1, model_, source, iteration, b, None, batchLabels)
            else:
                model_partial = feature_train(batchPaths, model1, model_, source, iteration, b, model_partial, batchLabels)

            b += 1

    elif source == 'target' and iteration != 0:   
        
        target_sample, labels_sample = target_sample_extract(n_k, pseudo_labels)
        
        b = 0
        
        for batchPaths, source_labels_ in source_batch_gen(batch_size, source_labels):
            
            assert(len(batchPaths) == len(source_labels_))
            
            if b == 0:
                
                batch_path = list(chain(batchPaths, target_sample))
                slabels = list(chain(source_labels_, labels_sample))
                batchLabels = np.array(slabels)
                
                assert(len(batch_path) == len(batchLabels))
                
                model_partial = feature_train(batch_path, model1, model_, source, iteration, b, None, batchLabels)
                
                b += 1
                
            elif b != 0:

                batchLabels = np.array(source_labels_)
                model_partial = feature_train(batchPaths, model1, model_, source, iteration, b, model_partial, batchLabels)
                
                b += 1

    del model1
    gc.collect()
    
    if source == 'target' and iteration != 0:
        return model_partial
    if source == 'source' and iteration == 0:
        return model_partial
    elif source == 'target' and iteration == 0:
        return np.hstack(np.asarray(pred))

In [ ]:
def sslda_online(M):
    
    score = []
    
    svm_model = feature_extract(step_size, None, labels, 'InceptionV3', 'source', None, 0, None)
    
    target_predictions = feature_extract(step_size, None, labels_t, 'InceptionV3', 'target', svm_model, 0, None)
    
    print(classification_report(labels_t, target_predictions, target_names = classes))

    for i in range(1, M):
        
        print('Iteration: ', i)
         
        n = (i / float(M)) * len(target_path_)
        
        if i == 1:
            mod = feature_extract(step_size, target_predictions, labels, 'InceptionV3', 'target', None, i, n)
            fin_pred = feature_extract(step_size, None, labels_t, 'InceptionV3', 'target', mod, 0, None)
        else:
            mod = feature_extract(step_size, fin_pred, labels, 'InceptionV3', 'target', None, i, n)
            fin_pred = feature_extract(step_size, None, labels_t, 'InceptionV3', 'target', mod, 0, None)
        
        score.append(accuracy_score(labels_t, fin_pred))
    
    del svm_model, target_predictions, mod
    gc.collect()
    
    return fin_pred, score

In [ ]:
def ensemble_sslda():
    
    pred = []
    mean_acc = []

    for i in range(20):

        print('ENSEMBLE ITERATION: ', i+1)

        sslda_pred, acc = sslda_online(10)

        pred.append(sslda_pred)
        mean_acc.append(acc)
        
        print(classification_report(labels_t, pred[-1], target_names = classes))
        
    return pred, mean_acc

In [ ]:
pred, mean_acc = ensemble_sslda()

In [ ]:
for id_, i in enumerate(mean_acc):
    plt.figure()
    plt.plot(i)
    plt.title('Accuracy over iterations')
    plt.xlabel('# Iterations')
    plt.ylabel('SVM mean accuracy')
    plt.show()

In [ ]:
for i in pred:
    print(classification_report(labels_t, i, target_names = classes))

In [ ]:
majority_vote = np.maximum.reduce([pred[0], pred[1], pred[2], pred[3], pred[4]])

In [ ]:
print(classification_report(labels_t, majority_vote, target_names = classes))